## Formulas 数据集预处理

### 一些 import 和设置

In [1]:
import os
import shutil

datasets_no_test 位置设置

In [2]:
datasets_no_test_path = './datasets_no_test/'

创建一些文件夹

In [10]:
def mkdir(path):
    folder = os.path.exists(path)
 
    if not folder:                   #判断是否存在文件夹如果不存在则创建为文件夹
        os.makedirs(path)            #makedirs 创建文件时如果路径不存在会创建这个路径

In [12]:
mkdir(datasets_no_test_path + "dev/images_after_filter")
mkdir(datasets_no_test_path + "dev/labels_after_filter")
mkdir(datasets_no_test_path + "dev/labels_after_tokenize")
mkdir(datasets_no_test_path + "train/images_after_filter")
mkdir(datasets_no_test_path + "train/labels_after_filter")
mkdir(datasets_no_test_path + "train/labels_after_tokenize")

### Tokenization，根据词表进⾏分词，并根据词表初步过滤数据

依据 no_chinese.py

指明路径

In [4]:
input_label_dirs = ['./datasets_no_test/dev/labels/', './datasets_no_test/train/labels/']
output_label_dirs = ['./datasets_no_test/dev/labels_after_tokenize/', './datasets_no_test/train/labels_after_tokenize/']

读取字符表

In [5]:
with open('./data_preprocess/vocab.txt', 'r', encoding='utf-8') as f:
    vocab = f.read().split()

max_token_len = 0
for v in vocab:
    if len(v) > max_token_len:
        # print(len(v))
        max_token_len = len(v)
# print(max_token_len)

搜索函数

In [6]:
def FMM_func(user_dict, sentence):
    """
    正向最大匹配（FMM）
    :param user_dict: 词典
    :param sentence: 句子
    """
    # 词典中最长词长度
    max_len = max([len(item) for item in user_dict])
    start = 0
    token_list = []
    while start != len(sentence):
        index = start+max_len
        if index>len(sentence):
            index = len(sentence)
        for i in range(max_len):
            if (sentence[start:index] in user_dict) or (len(sentence[start:index])==1):
                token_list.append(sentence[start:index])
                # print(sentence[start:index], end='/')
                start = index
                break
            index += -1
    return token_list

处理函数

In [7]:
chinese_token_list=[]

def tokenize(input_label_dir, output_label_dir):
    label_name_list = os.listdir(input_label_dir)
    index = 1
    for label_name in label_name_list:
        # print(index, ':')
        index += 1
        # print(label_name, ':',end='')
        label_file_name = input_label_dir + label_name
        with open(label_file_name, 'r', encoding='utf-8') as f1:
            content = f1.read()

        # print(content)

        token_list = FMM_func(vocab, content)
        token_list = [token_list[i] for i in range(len(token_list)) if token_list[i] != ' '] # 去除空格
        # print(token_list)

        new_content = ' '.join(token_list)

        # print(new_content)
        
        have_chinese = False

        for token in token_list:
            if token not in vocab and token not in ['', ' ']:
                # print(label_name, ':',end='')
                # print(token)
                chinese_token_list.append(token)
                have_chinese = True

        if have_chinese is not True:
            # 保存数据
            # shutil.copy(label_file_name, output_label_dir + label_name)
            with open(output_label_dir + label_name, 'w', encoding='utf-8') as f:
                f.write(new_content)
        else:
            # 不管了
            pass
            # with open('./data/math_210421/formula_labels_210421_chinese/' + label_name, 'w', encoding='utf-8') as f:
                # f.write(new_content)

        # if have_chinese is True:
        #     print()

运行处理

In [13]:
for input_label_dir, output_label_dir in zip(input_label_dirs, output_label_dirs): 
    tokenize(input_label_dir, output_label_dir)

保存出现的中文

In [14]:
with open('./data_preprocess/chinese_token.txt', 'w', encoding='utf-8') as f:
    chinese_token_list = list(set(chinese_token_list))
    for chinese_token in chinese_token_list:
        f.write(chinese_token + '\n')

### 过滤多行和内容为error mathpix的标签

依据 data_filter.py

指明路径

In [15]:
input_label_dirs = ['./datasets_no_test/dev/labels_after_tokenize/', './datasets_no_test/train/labels_after_tokenize/']
output_label_dirs = ['./datasets_no_test/dev/labels_after_filter/', './datasets_no_test/train/labels_after_filter/']

过滤函数

In [16]:
def filter(input_label_dir, output_label_dir):
    label_name_list = os.listdir(input_label_dir)
    for label_name in label_name_list:
        label_file_name = input_label_dir + label_name
        with open(label_file_name, 'r', encoding='utf-8') as f1:
            lines = f1.readlines()
            content = f1.read()

        # print(lines)
        # print(content)

        if len(lines) > 1 or 'error mathpix' in content:
            # 多行和错误的数据直接不要了
            continue

        # 通过筛选的数据
        shutil.copy(label_file_name, output_label_dir + label_name)

运行处理

In [17]:
for input_label_dir, output_label_dir in zip(input_label_dirs, output_label_dirs): 
    filter(input_label_dir, output_label_dir)

### 对⻬过滤后的数据

依据 extract_image_according_to_label_list.py

指明路径

In [21]:
label_dirs = ['./datasets_no_test/dev/labels_after_filter/', './datasets_no_test/train/labels_after_filter/']
image_dirs = ['./datasets_no_test/dev/images/', './datasets_no_test/train/images/']
output_dirs = ['./datasets_no_test/dev/images_after_filter/', './datasets_no_test/train/images_after_filter/']

处理函数

In [19]:
def extract(label_dir, image_dir, output_dir):
    label_name_list = os.listdir(label_dir)

    for i in range(len(label_name_list)):
        label_name_list[i] = label_name_list[i][:-4]

    # print(label_list)

    image_name_list = os.listdir(image_dir)

    for image_name in image_name_list:
        if image_name[:-4] in label_name_list:
            # print(image_name)
            shutil.copy(image_dir + image_name, output_dir + image_name)

运行处理

In [22]:
for label_dir, image_dir, output_dir in zip(label_dirs, image_dirs, output_dirs):
    extract(label_dir, image_dir, output_dir)

1.png
10016.png
10044.png
10061.png
10081.png
10090.png
10099.png
10113.png
10119.png
10128.png
10130.png
10131.png
10143.png
10144.png
10156.png
10159.png
10172.png
10182.png
10190.png
10192.png
10195.png
10205.png
10209.png
10221.png
10225.png
10228.png
10235.png
10244.png
10256.png
10262.png
10266.png
10267.png
10295.png
10298.png
10299.png
10301.png
10324.png
10329.png
10332.png
10333.png
10335.png
10339.png
10347.png
10349.png
10354.png
10362.png
10365.png
1038.png
10397.png
10399.png
1040.png
10400.png
10412.png
10421.png
10432.png
10441.png
1046.png
10462.png
10465.png
10469.png
1047.png
10471.png
10477.png
10479.png
10481.png
10488.png
10496.png
10497.png
105.png
1051.png
1052.png
10542.png
10550.png
10554.png
10563.png
10564.png
10582.png
10598.png
10613.png
10615.png
10616.png
10619.png
1062.png
10624.png
10626.png
10627.png
10631.png
1065.png
10653.png
10654.png
10658.png
10659.png
10664.png
10670.png
10674.png
10679.png
10695.png
10715.png
1072.png
10721.png
10723.png
10730

### 根据神经⽹络模型的需要，进行padding

In [2]:
# TODO